In [1]:
# # RUN BELOW WHEN RUNNING FOR THE FIRST TIME
# import nltk
# nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('wordnet')

### Core Assumptions:
* Business wants to classify incoming textual data
* Minimal labeling needed. Zero to 20 labels per classification 

In [1]:
import pandas as pd
import random
from topic_modeling_pipeline import *
from classification_pipeline import *
random.seed(42)

In [2]:
taking_sample = True
test_sample_size = 30

## AG News Dataset

In [3]:
ag_test = pd.read_csv('../data/AG News/test.csv')
ag_train = pd.read_csv('../data/AG News/train_from_llm.csv')

ag_train.dropna(subset=['Predicted Topic Index'], inplace=True)

# CREATING A SAMPLE TEST SET
if taking_sample == True:
    ag_test = ag_test.sample(test_sample_size, random_state=42)

ag_test_desc = ag_test['Description']
ag_test_labels = ag_test['Class Index']

ag_train_desc = ag_train['Description']
ag_train_labels = ag_train['Class Index']

ag_train_llm_desc = ag_train['Description']
ag_train_llm_labels = ag_train['Predicted Topic Index']

### BBC

In [4]:
bbc_test = pd.read_csv('../data/BBC News/test.csv')
bbc_train = pd.read_csv('../data/BBC News/train_from_llm.csv')

if taking_sample == True:
    bbc_test = bbc_test.sample(test_sample_size, random_state=42)

bbc_test_desc = bbc_test['Description']
bbc_test_labels = bbc_test['Class Index']

bbc_train_desc = bbc_train['Description']
bbc_train_labels = bbc_train['Class Index']

bbc_train_llm_desc = ag_train['Description']
bbc_train_llm_labels = ag_train['Predicted Topic Index']


### 20NewsGroup

In [5]:
newsgroups_test = pd.read_csv('../data/20 Newsgroups/test.csv')
newsgroups_train = pd.read_csv('../data/20 Newsgroups/train_from_llm.csv')
newsgroups_train = newsgroups_train[newsgroups_train['Predicted Topic Index'] != -1]

if taking_sample == True:
    newsgroups_test = newsgroups_test.sample(test_sample_size, random_state=42)

newsgroups_test_desc = newsgroups_test['Description']
newsgroups_test_labels = newsgroups_test['Class Index']

newsgroups_train_desc = newsgroups_train['Description']
newsgroups_train_labels = newsgroups_train['Class Index']

newsgroups_train_llm_desc = newsgroups_train['Description']
newsgroups_train_llm_labels = newsgroups_train['Predicted Topic Index']

### Combining Datasets

In [6]:
# len(bbc_test_desc), len(bbc_train_desc), len(ag_test_desc), len(ag_train_desc)

In [8]:
# For pipelines that works with all datasets
datasets_test = {
    "AG News Full": (ag_test_desc, ag_test_labels),
    "AG News LLM labels": (ag_test_desc, ag_test_labels),
    "BBC News Full": (bbc_test_desc, bbc_test_labels),
    "BBC News LLM labels": (bbc_test_desc, bbc_test_labels),
    "20 Newsgroups Full": (newsgroups_test_desc, newsgroups_test_labels),
    "20 Newsgroups LLM labels": (newsgroups_test_desc, newsgroups_test_labels),
}

datasets_train = {
    "AG News Full":(ag_train_desc, ag_train_labels),
    "AG News LLM labels":(ag_train_llm_desc, ag_train_llm_labels),
    "BBC News Full": (bbc_train_desc, bbc_train_labels),
    "BBC News LLM labels": (bbc_train_llm_desc, bbc_train_llm_labels),
    "20 Newsgroups Full": (newsgroups_train_desc, newsgroups_train_labels),
    "20 Newsgroups LLM labels": (newsgroups_train_llm_desc, newsgroups_train_llm_labels),
}

# For pipelines that works with AG News only
datasets_test_ag_news = {
    "AG News Full": (ag_test_desc, ag_test_labels),
    "AG News LLM labels": (ag_test_desc, ag_test_labels),
}

datasets_train_ag_news = {
    "AG News Full":(ag_train_desc, ag_train_labels),
    "AG News LLM labels":(ag_train_llm_desc, ag_train_llm_labels),
}

# For pipelines that works with BBC News only
datasets_test_bbc_news = {
    "BBC News Full": (bbc_test_desc, bbc_test_labels),
    "BBC News LLM labels": (bbc_test_desc, bbc_test_labels),
}

datasets_train_bbc_news = {
    "BBC News Full": (bbc_train_desc, bbc_train_labels),
    "BBC News LLM labels": (bbc_train_llm_desc, bbc_train_llm_labels),
}

datasets_newsgroups = {
    "20 Newsgroups Full": (newsgroups_test_desc, newsgroups_test_labels),
    "20 Newsgroups LLM labels": (newsgroups_test_desc, newsgroups_test_labels),
}

datasets_train_newsgroups = {
    "20 Newsgroups Full": (newsgroups_train_desc, newsgroups_train_labels),
    "20 Newsgroups LLM labels": (newsgroups_train_llm_desc, newsgroups_train_llm_labels),
}

In [9]:
topic_modelling_orchestrator_ag_news = TopicModelingPipelineOrchestrator()
topic_modelling_orchestrator_bbc_news = TopicModelingPipelineOrchestrator()
topic_modelling_orchestrator_ag_news.add_models_grid(
    model_types=[
        'LDA', 
        'LSI',
        'NMF'
        ],
    param_grid={'n_topics': [4]}
)
topic_modelling_orchestrator_bbc_news.add_models_grid(
    model_types=[
        'LDA', 
        'LSI', 
        'NMF'
        ],
    param_grid={'n_topics': [5]}
)

['LDA_5', 'LSI_5', 'NMF_5']

In [10]:
class_orchestrator_zero_shot = ClassificationPipelineOrchestrator()
class_orchestrator_sup = ClassificationPipelineOrchestrator()
class_orchestrator_zero_shot.add_models_grid(
    model_types=[
        'LLMClassifier',
        'TARSZeroShot',
    ]
)
class_orchestrator_sup.add_models_grid(
    model_types=[
        'SVM',
        'XGBoost',
        'LightGBM',
        'RandomForest',
        'SVMRoberta', 
        'XGBoostRoberta',
        'LightGBMRoberta',
        'RandomForestRoberta'
    ]
    # param_grid={'SVMRoberta': [{}], 'SVM': [{}]}  # Empty dictionary means default parameters
)

2025-06-05 22:33:37,184 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model


['SVM',
 'XGBoost',
 'LightGBM',
 'RandomForest',
 'SVMRoberta',
 'XGBoostRoberta',
 'LightGBMRoberta',
 'RandomForestRoberta']

In [11]:
from noise_strategy import *
noise_strategies = [
    NoNoise(),
    # AddRandomCharsNoise(), 
    # AddRandomWordsNoise(), 
    # DeleteRandomWordsNoise(), 
    # ShuffleSentencesNoise(noise_level=0.7), 
    # ReplaceWithSynonymsNoise(), 
    # ReplaceWithAntonymsNoise()
]

#### Running pipelines for classification

In [12]:
class_orchestrator_sup.evaluate_with_training(datasets_train, datasets_test, noise_strategies).round(2)

Datasets:   0%|          | 0/6 [00:00<?, ?it/s]

2025-06-05 22:33:40,792 [INFO] root: Processing dataset: AG News Full


Models:   0%|          | 0/8 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

...

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Dataset,Noise,Model,Accuracy,F1 Score,Precision,Recall
34,20 Newsgroups Full,NoNoise,LightGBM,0.10,0.12,0.17,0.10
38,20 Newsgroups Full,NoNoise,LightGBMRoberta,0.47,0.45,0.50,0.47
35,20 Newsgroups Full,NoNoise,RandomForest,0.37,0.41,0.72,0.37
39,20 Newsgroups Full,NoNoise,RandomForestRoberta,0.47,0.47,0.53,0.47
32,20 Newsgroups Full,NoNoise,SVM,0.37,0.39,0.53,0.37
36,20 Newsgroups Full,NoNoise,SVMRoberta,0.50,0.49,0.52,0.50
33,20 Newsgroups Full,NoNoise,XGBoost,0.30,0.30,0.42,0.30
37,20 Newsgroups Full,NoNoise,XGBoostRoberta,0.50,0.50,0.58,0.50
42,20 Newsgroups LLM labels,NoNoise,LightGBM,0.10,0.11,0.13,0.10
46,20 Newsgroups LLM labels,NoNoise,LightGBMRoberta,0.33,0.33,0.37,0.33


In [15]:
class_orchestrator_sup.results.round(2).to_csv('../outputs/final/results_classification_supervised.csv', index=False)

In [13]:
class_orchestrator_zero_shot.evaluate_with_training(datasets_train, datasets_test, noise_strategies).round(2)

Datasets:   0%|          | 0/6 [00:00<?, ?it/s]

2025-06-05 22:37:08,534 [INFO] root: Processing dataset: AG News Full


Models:   0%|          | 0/2 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Classifying with LLM:   0%|          | 0/30 [00:00<?, ?it/s]

...

Classifying with LLM:   0%|          | 0/30 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Dataset,Noise,Model,Accuracy,F1 Score,Precision,Recall
8,20 Newsgroups Full,NoNoise,LLMClassifier,0.57,0.55,0.59,0.57
9,20 Newsgroups Full,NoNoise,TARSZeroShot,0.07,0.09,0.13,0.07
10,20 Newsgroups LLM labels,NoNoise,LLMClassifier,0.57,0.55,0.59,0.57
11,20 Newsgroups LLM labels,NoNoise,TARSZeroShot,0.07,0.09,0.13,0.07
0,AG News Full,NoNoise,LLMClassifier,0.90,0.90,0.92,0.90
1,AG News Full,NoNoise,TARSZeroShot,0.80,0.75,0.72,0.80
2,AG News LLM labels,NoNoise,LLMClassifier,0.90,0.90,0.92,0.90
3,AG News LLM labels,NoNoise,TARSZeroShot,0.80,0.75,0.72,0.80
4,BBC News Full,NoNoise,LLMClassifier,0.90,0.89,0.91,0.90
5,BBC News Full,NoNoise,TARSZeroShot,0.83,0.82,0.85,0.83


In [17]:
class_orchestrator_zero_shot.results.round(2).to_csv('../outputs/final/results_classification_zero_shot.csv', index=False)

# UP TO HERE WORKS WELL, CAN RUN ON ALL DATASETS

#### Running pipelines for Topic Modeling

In [18]:
topic_modelling_orchestrator_ag_news.evaluate(datasets_test_ag_news, noise_strategies)
topic_modelling_orchestrator_ag_news.results

Datasets:   0%|          | 0/2 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

,Dataset,Noise,Model,ARI Score,Topics Coherence,Cosine Similarity,Reconstruction Error
0,AG News Full,NoNoise,LDA_4,0.411765,0.944541,0.364855,None
1,AG News Full,NoNoise,LSI_4,0.411765,0.821863,0.280000,None
2,AG News Full,NoNoise,NMF_4,0.411765,0.901785,0.280000,0.988373
3,AG News LLM labels,NoNoise,LDA_4,0.411765,0.944541,0.364855,None
4,AG News LLM labels,NoNoise,LSI_4,0.411765,0.821863,0.280000,None
5,AG News LLM labels,NoNoise,NMF_4,0.411765,0.901785,0.280000,0.988373


In [20]:
topic_modelling_orchestrator_ag_news.results.round(2).to_csv('../outputs/final/results_topic_modelling_ag_news.csv', index=False)

In [21]:
topic_modelling_orchestrator_bbc_news.evaluate(datasets_test_bbc_news, noise_strategies)
topic_modelling_orchestrator_bbc_news.results

Datasets:   0%|          | 0/1 [00:00<?, ?it/s]

Models:   0%|          | 0/3 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

Noise Strategies:   0%|          | 0/1 [00:00<?, ?it/s]

,Dataset,Noise,Model,ARI Score,Topics Coherence,Cosine Similarity,Reconstruction Error
0,BBC News Full,NoNoise,LDA_5,-0.25,0.479341,0.393866,None
1,BBC News Full,NoNoise,LSI_5,0.00,0.734039,0.219998,None
2,BBC News Full,NoNoise,NMF_5,0.00,0.918731,0.200008,0.000122


In [22]:
topic_modelling_orchestrator_bbc_news.results.round(2).to_csv('../outputs/final/results_topic_modelling_bbc_news.csv', index=False)
